## 导入节点数据并对其进行信息重构

In [2]:
from itertools import chain
import itertools
from collections import defaultdict
import re

def lines_per_n(f, n):
    for line in f:
        yield ''.join(chain([line], itertools.islice(f, n - 1)))
## 定义节点信息字典——打开raw_data下的nodelist
node_data = defaultdict(lambda : ())  
with open('C:\\Users\\Wei Zhou\\Desktop\\test\\图神经网络几个算法\\动态图-DySAT\\DySAT_pytorch-main\\raw_data\\Enron\\vis.graph.nodeList.json') as f:
    for chunk in lines_per_n(f, 5):
        chunk = chunk.split("\n")
    ## 遍历每个chunk获取其中的id    
        id_string = chunk[1].split(":")[1]
        x = [x.start() for x in re.finditer('\"', id_string)]
        id =  id_string[x[0]+1:x[1]]
    ## 遍历每个chunk获取其中的name    
        name_string = chunk[2].split(":")[1]
        x = [x.start() for x in re.finditer('\"', name_string)]
        name =  name_string[x[0]+1:x[1]]
    ## 遍历每个chunk获取其中的idx        
        idx_string = chunk[3].split(":")[1]
        x1 = idx_string.find('(')
        x2 = idx_string.find(')')
        idx =  idx_string[x1+1:x2]
            
        print("ID:{}, IDX:{:<4}, NAME:{}".format(id, idx, name))
    ##给节点编号
        node_data[name] = (id,idx)
        
print(len(node_data))

ID:55098b62251497209062421f, IDX:0   , NAME:albert.meyers@enron.com
ID:55098b622514972090624220, IDX:1   , NAME:andrea.ring@enron.com
ID:55098b622514972090624221, IDX:2   , NAME:andrew.lewis@enron.com
ID:55098b622514972090624222, IDX:3   , NAME:andy.zipper@enron.com
ID:55098b622514972090624223, IDX:4   , NAME:barry.tycholiz@enron.com
ID:55098b622514972090624224, IDX:5   , NAME:benjamin.rogers@enron.com
ID:55098b622514972090624225, IDX:6   , NAME:bill.rapp@enron.com
ID:55098b622514972090624226, IDX:7   , NAME:bill.williams@enron.com
ID:55098b622514972090624227, IDX:8   , NAME:brad.mckay@enron.com
ID:55098b622514972090624228, IDX:9   , NAME:brenda.whitehead@enron.com
ID:55098b622514972090624229, IDX:10  , NAME:cara.semperger@enron.com
ID:55098b62251497209062422a, IDX:11  , NAME:charles.weldon@enron.com
ID:55098b62251497209062422b, IDX:12  , NAME:chris.dorland@enron.com
ID:55098b62251497209062422c, IDX:13  , NAME:chris.germany@enron.com
ID:55098b62251497209062422d, IDX:14  , NAME:clint.de

## 导入边数据并对其进行信息重构

In [9]:
import dateutil.parser
def getDateTimeFromISO8601String(s):
    d = dateutil.parser.parse(s)
    return d

##边连接列表
links = []
##时间戳列表
ts = []
with open('C:\\Users\\Wei Zhou\\Desktop\\test\\图神经网络几个算法\\动态图-DySAT\\DySAT_pytorch-main\\raw_data\\Enron\\vis.digraph.allEdges.json') as f:
    for chunk in lines_per_n(f, 5):
        chunk = chunk.split("\n")
        ##获取边连接    
        name_string = chunk[2].split(":")[1]
        x = [x.start() for x in re.finditer('\"', name_string)]
        from_id, to_id = name_string[x[0]+1:x[1]].split("_")
        ##获取时间戳    
        time_string = chunk[3].split("ISODate")[1]
        x = [x.start() for x in re.finditer('\"', time_string)]
        timestamp = getDateTimeFromISO8601String(time_string[x[0]+1:x[1]])
        ##添加进列表
        ts.append(timestamp)
        links.append((from_id, to_id, timestamp))
        
#对时间进行排序
links.sort(key =lambda x: x[2])
print(links)

[('mark.taylor@enron.com', 'tana.jones@enron.com', datetime.datetime(1998, 11, 13, 12, 7, tzinfo=tzutc())), ('mark.taylor@enron.com', 'tana.jones@enron.com', datetime.datetime(1998, 11, 13, 12, 7, tzinfo=tzutc())), ('mark.taylor@enron.com', 'michelle.cash@enron.com', datetime.datetime(1998, 11, 19, 15, 19, tzinfo=tzutc())), ('mark.taylor@enron.com', 'michelle.cash@enron.com', datetime.datetime(1998, 11, 19, 15, 19, tzinfo=tzutc())), ('mark.taylor@enron.com', 'marie.heard@enron.com', datetime.datetime(1998, 11, 24, 10, 23, tzinfo=tzutc())), ('mark.taylor@enron.com', 'marie.heard@enron.com', datetime.datetime(1998, 11, 24, 10, 23, tzinfo=tzutc())), ('mark.taylor@enron.com', 'richard.sanders@enron.com', datetime.datetime(1998, 12, 2, 11, 47, tzinfo=tzutc())), ('mark.taylor@enron.com', 'richard.sanders@enron.com', datetime.datetime(1998, 12, 2, 11, 47, tzinfo=tzutc())), ('mark.taylor@enron.com', 'tana.jones@enron.com', datetime.datetime(1998, 12, 2, 13, 24, tzinfo=tzutc())), ('mark.taylor@

## 划分快照时间段

In [12]:
from datetime import timedelta
import networkx as nx
## 按照每两月进行一次快照拍摄
SLICE_MONTHS = 2
## 确定起止日期（可调节时间段）
START_DATE = min(ts) + timedelta(200)
END_DATE = max(ts) - timedelta(200)

##创建快照切片的连接，即获得每两个月进行一次快照的多图合集
slice_links = defaultdict(lambda: nx.MultiGraph())

##遍历边列表
for (a, b, time) in links:
    datetime_object = time
    ##当遍历的边列表中的发生交互时间超过规定的最大时间按照最大时间进行计算
    if datetime_object > END_DATE:
        ##计算时间单位，从开始的天转化为月
        months_diff = (END_DATE - START_DATE).days//30
    else:
        months_diff = (datetime_object - START_DATE).days//30
        
    ##进一步精确时间单位为2个月
    slice_id = months_diff // SLICE_MONTHS
    slice_id = max(slice_id, 0)

    ##创建在以2个月为时间单位下产生的时间快照的集合
    if slice_id not in slice_links.keys():
    ##如果该时刻不存在，建立该时刻的graph
        slice_links[slice_id] = nx.MultiGraph()
        ##将前一时间快照的节点放入该时间快照图中
        if slice_id > 0:
            slice_links[slice_id].add_nodes_from(slice_links[slice_id-1].nodes(data=True))
            assert (len(slice_links[slice_id].edges()) ==0)
    ##利用之前的边列表为对应时间的节点之间的添加边连接
    slice_links[slice_id].add_edge(a,b, date=datetime_object)

print(slice_links)

defaultdict(<function <lambda> at 0x00000253D2811000>, {0: <networkx.classes.multigraph.MultiGraph object at 0x00000253D01FF400>, 1: <networkx.classes.multigraph.MultiGraph object at 0x00000253D01FD990>, 2: <networkx.classes.multigraph.MultiGraph object at 0x00000253D2406C80>, 3: <networkx.classes.multigraph.MultiGraph object at 0x00000253D2406DD0>, 4: <networkx.classes.multigraph.MultiGraph object at 0x00000253D01FE0B0>, 5: <networkx.classes.multigraph.MultiGraph object at 0x00000253D2139E10>, 6: <networkx.classes.multigraph.MultiGraph object at 0x00000253D213A170>, 7: <networkx.classes.multigraph.MultiGraph object at 0x00000253D212A5F0>, 8: <networkx.classes.multigraph.MultiGraph object at 0x00000253D212A5C0>, 9: <networkx.classes.multigraph.MultiGraph object at 0x00000253D2129F30>, 10: <networkx.classes.multigraph.MultiGraph object at 0x00000253D212A290>, 11: <networkx.classes.multigraph.MultiGraph object at 0x00000253D212A200>, 12: <networkx.classes.multigraph.MultiGraph object at 

In [13]:
## 获取每个图的静态信息
used_nodes = []
## 遍历每个图
for id, slice in slice_links.items():
    print("In snapshoot {:<2}, #Nodes={:<5}, #Edges={:<5}".format(id, \
slice.number_of_nodes(), slice.number_of_edges()))
##遍历图中的每个节点
    for node in slice.nodes():
            if not node in used_nodes:
            ##全部节点信息
                used_nodes.append(node)
        
## 添加连续性的索引    
nodes_consistent_map = {node:idx for idx, node in enumerate(used_nodes)}

##并用索引重新标记图中的节点
for id, slice in slice_links.items():
        slice_links[id] = nx.relabel_nodes(slice, nodes_consistent_map)


In snapshoot 0 , #Nodes=18   , #Edges=237  
In snapshoot 1 , #Nodes=23   , #Edges=184  
In snapshoot 2 , #Nodes=24   , #Edges=216  
In snapshoot 3 , #Nodes=50   , #Edges=570  
In snapshoot 4 , #Nodes=66   , #Edges=649  
In snapshoot 5 , #Nodes=79   , #Edges=877  
In snapshoot 6 , #Nodes=98   , #Edges=1273 
In snapshoot 7 , #Nodes=110  , #Edges=2164 
In snapshoot 8 , #Nodes=117  , #Edges=2688 
In snapshoot 9 , #Nodes=125  , #Edges=2612 
In snapshoot 10, #Nodes=131  , #Edges=2601 
In snapshoot 11, #Nodes=135  , #Edges=2852 
In snapshoot 12, #Nodes=137  , #Edges=1448 
In snapshoot 13, #Nodes=138  , #Edges=581  
In snapshoot 14, #Nodes=141  , #Edges=2125 
In snapshoot 15, #Nodes=143  , #Edges=1707 


In [19]:
from scipy.sparse import csr_matrix
import numpy as np
##选取最后一个图中的所有节点，为每个节点建立one-hot编码（因为每次图的添入会加入新产生的节点，所以最后一张图是包含了所有的节点的，然后对此进行编码）
onehot = np.identity(slice_links[max(slice_links.keys())].number_of_nodes())
graphs = []

## 遍历快照集合
for id, slice in slice_links.items():
    tmp_feature = []
    ##遍历该快照下的节点并且将涉及到的节点的特征导入从而生成该快照下的节点特征
    for node in slice.nodes():
        tmp_feature.append(onehot[node])
    ##建立新的快照信息（节点特征）并将其导入
    slice.graph["feature"] = csr_matrix(tmp_feature)
    graphs.append(slice)
    
print(graphs[-1].graph["feature"].A.shape)

(143, 143)


## 储存图信息

In [ ]:
import pickle as pkl
save_path = "../../data/Enron/graph.pkl"
with open(save_path, "wb") as f:
    pkl.dump(graphs, f)
print("Processed Data Saved at {}".format(save_path))